# Quelle est la page précédente des visiteurs des pages info licenciement

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict
tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "range": {
            "logfile": {
              "gte": "2023-02-01",
              "lt": "2023-04-01"
            }
          }
        }
      ]
    }
  }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
#pour avoir à éviter de charger les données
#logs.to_csv('../../save_fev_marc_data.csv', index=False)

# puis au chargement
#logs = pd.read_csv('../../save_fev_marc_data.csv')


In [ ]:
logs.shape

In [ ]:
logs.head()

In [ ]:
logs.type.value_counts()

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
def clean_dataset_from_useless_rows(df):
    df_shortened = df[df['type'].isin(['visit_content', 'outil', 'themes', 'page_information',
                                       'search', 'home', 'modeles_courriers', 'outlink'])]
    return df_shortened.loc[
        (df_shortened['idVisit'].shift() != df_shortened['idVisit']) |
        (df_shortened['url'].shift() != df_shortened['url']) |
        #(df_shortened['type'].shift() != df_shortened['type']) |
        (df_shortened['referrerTypeName'].shift() != df_shortened['referrerTypeName'])
    ]

In [ ]:
logs_formated = format_url_and_get_month_and_datetime_columns(logs.copy())
logs_cleaned = clean_dataset_from_useless_rows(logs_formated.copy())

In [ ]:
logs_cleaned.shape[0]

# Début de l'explo

On va s'y prendre en plusieurs étapes. 
- Supprimons des données tous les visiteurs qui NE sont PAS allés sur les pages info licenciement
- Ordonnons par IdVisit et timestamp nos données
- Débarasser des logs inutiles, nous allons dupliquer la colonne url en une colonne url_précédent en la décalant de 1 ligne (en gros l'url précédent est égal à l'url actuel de la ligne précédente)
    - cela va créer un souci car le premier url de visite d'un utilisateur sera le dernier url de l'utilisateur précédent. On pourra facilement corriger ça en modifiant le premier url_précédent de chaque utilisateur par le referrer_name de la ligne actuelle. De telle sorte à ce que la première ligne contienne le point d'entrée de l'utilisateur et les lignes suivantes soient composé de l'url de la ligne précédente. Est-ce que c'est clair ? 😅 
- Ensuite on voit ce qu'on fait mais on pourrait faire un groupby url et url précédent et count occurences 

### 1. Suppressions des visiteurs qui NE sont PAS allés sur les pages info licenciement

In [ ]:
# On priviligie tous les logs pour ne manquer un utilisateur
visitors_having_visited_info_licenc = set(logs[
    (logs['url'].notna()) & 
    (logs['url'].str.contains('information/')) &
    (logs['url'].str.contains('licenciement'))
].idVisit.unique())

len(visitors_having_visited_info_licenc)

In [ ]:
logs_visitors_having_visited_info_licenc = logs_cleaned[
    logs_cleaned['idVisit'].isin(visitors_having_visited_info_licenc)
]

### 2. Ordonnons par IdVisit et timestamp nos données

In [ ]:
logs_sorted = logs_visitors_having_visited_info_licenc.sort_values(by=['idVisit', 'timestamp'])

### 3. On ajoute une colonne url_précédent

In [ ]:
logs_sorted['url_précédent'] = logs_sorted['url'].shift()

In [ ]:
logs_sorted.head(5)

In [ ]:
def update_first_url_precedent_with_refererrName(df):
    df.iloc[0, df.columns.get_loc('url_précédent')] = df.iloc[0].referrerName
    return df

In [ ]:
logs_with_previous_url = logs_sorted\
    .groupby(by='idVisit', group_keys=False)\
    .progress_apply(update_first_url_precedent_with_refererrName)

In [ ]:
logs_with_previous_url

### 4. Ensuite on voit ce qu'on fait mais on pourrait faire un groupby url et url précédent et count occurences 